In [4]:
import polars as pl
from pathlib import Path

# sample data frame
file_name = Path("./sample.parquet")
data = {"a": [1, 2], "b": [3, 4]}
df = pl.DataFrame(data)
df.write_parquet(file_name)
df

a,b
i64,i64
1,3
2,4


In [4]:
|file_name

PosixPath('sample.parquet')

In [2]:
from src.core_pro import AWS

bucket_name = 'sg-vnbi-ops-kevin'
s3 = AWS(bucket_name)
# response = s3.upload_single_file(file=file_name, prefix="sample")

In [8]:
flink, sql = s3.convert_dataframe_flink_config(df, "sample", "dev_vnbi_ops.ds_cx__item_marketplace_listening__s3_adhoc")

In [10]:
print(flink)


env {
}      

source {
    S3FileSource {
        datasource_name = "my_s3"
        path = "s3a://sg-vnbi-ops-kevin/sample"
        format = "parquet"
        schema = {
a:BIGINT
, b: BIGINT
}
        endpoint = "https://s3g.data-infra.shopee.io"
        bucket = "sg-vnbi-ops-kevin"
        # ignore-parse-errors = "true"
        # skip-first-data-row = "true"
        result_table_name = "s3FileSource"
        access_key = "${HADOOP_USER_NAME}"
        secret_key = "${HADOOP_USER_RPCPASSWORD}"
        endpoint = "https://s3g.data-infra.shopee.io"
        fs.s3a.change.detection.version.required = "false"
      }
}

transform {
  Sql {
    sql = "select * from `s3FileSource`"
    result_table_name = "transformed"
  }
}
  
sink {
  HiveSink {
    source_table_name = "transformed"
    result_table_name = "hive.dev_vnbi_ops.ds_cx__item_marketplace_listening__s3_adhoc"
    partition_by = ["grass_date"]
    partition_dir_expression = "grass_date=${grass_date}"
  }
}
        


In [11]:
print(sql)


drop table if exists dev_vnbi_ops.ds_cx__item_marketplace_listening__s3_adhoc
;

create table if not exists table_name (
    a bigint
	, b bigint
    ,`grass_date` date
)
partitioned by
    (`grass_date`) stored as PARQUET location 'ofs://oc/s3v/sg-vnbi-ops-hive/dev_vnbi_ops/ds_cx__item_marketplace_listening__s3_adhoc'
;        

        
